In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd

# Substitua o 'caminho_do_arquivo.csv' pelo caminho real do seu arquivo CSV
arquivo_csv = '/content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic/data/hate_speech_data_paula.csv'

# Ler o arquivo CSV
dados = pd.read_csv(arquivo_csv)

# Mostrar as primeiras linhas do DataFrame
print(dados.head())


                                                text  hatespeech_comb
0  @__andrea__b \nO cara vive em outro mundo\nNão...                1
1  @_carmeloneto Estes incompetentes não cuidam n...                0
2  @_carmeloneto \nOs 'cumpanhero' quebraram toda...                0
3  @_GlitteryKisses é isso não conseguem pensar n...                0
4                @_iglira bom dia macaco branco haha                1


In [6]:
# Selecionar colunas específicas
dados_brutos = dados.iloc[:, 0]
rotulos = dados.iloc[:, 1]

# Mostrar as primeiras linhas de cada coluna
print("Dados brutos:")
print(dados_brutos.head())
print("\nRótulos:")
print(rotulos.head())


Dados brutos:
0    @__andrea__b \nO cara vive em outro mundo\nNão...
1    @_carmeloneto Estes incompetentes não cuidam n...
2    @_carmeloneto \nOs 'cumpanhero' quebraram toda...
3    @_GlitteryKisses é isso não conseguem pensar n...
4                  @_iglira bom dia macaco branco haha
Name: text, dtype: object

Rótulos:
0    1
1    0
2    0
3    0
4    1
Name: hatespeech_comb, dtype: int64


In [9]:
# Avalia com modelo simples
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

# Substitua o 'caminho_do_arquivo_treino.csv' pelo caminho real do seu arquivo CSV de treino
arquivo_csv_treino = '/content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic/data/hate_speech_data_paula.csv'
dados_treino = pd.read_csv(arquivo_csv_treino)

# Pré-processamento dos dados de treino
X = dados_treino.iloc[:, 0]  # Coluna dos dados preprocessados
y = dados_treino.iloc[:, 1]  # Coluna dos rótulos

# Codificação dos rótulos para 0 e 1
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Tokenização e sequenciamento dos textos
max_length = 280  # Define o tamanho máximo das sequências
vocab_size = 100000  # Define o tamanho do vocabulário

tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

# Dividir os dados em conjuntos de treino e validação
X_train, X_val, y_train, y_val = train_test_split(padded_sequences, y, test_size=0.2, random_state=42)

# Definir o modelo
embedding_dim = 16

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compilar o modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Treinar o modelo
epochs = 10
history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val), verbose=2)

# Avaliação do modelo nos dados de teste
loss, accuracy = model.evaluate(X_val, y_val, verbose=2)
print(f"Perda nos dados de teste: {loss:.4f}")
print(f"Acurácia nos dados de teste: {accuracy:.4f}")


Epoch 1/10
142/142 - 3s - loss: 0.6261 - accuracy: 0.6814 - val_loss: 0.6062 - val_accuracy: 0.6931 - 3s/epoch - 21ms/step
Epoch 2/10
142/142 - 2s - loss: 0.5561 - accuracy: 0.6909 - val_loss: 0.5492 - val_accuracy: 0.7372 - 2s/epoch - 15ms/step
Epoch 3/10
142/142 - 2s - loss: 0.3727 - accuracy: 0.8486 - val_loss: 0.5627 - val_accuracy: 0.7354 - 2s/epoch - 13ms/step
Epoch 4/10
142/142 - 2s - loss: 0.1964 - accuracy: 0.9669 - val_loss: 0.5773 - val_accuracy: 0.7240 - 2s/epoch - 13ms/step
Epoch 5/10
142/142 - 2s - loss: 0.0477 - accuracy: 0.9956 - val_loss: 0.6525 - val_accuracy: 0.7108 - 2s/epoch - 13ms/step
Epoch 6/10
142/142 - 2s - loss: 0.0154 - accuracy: 0.9991 - val_loss: 0.7084 - val_accuracy: 0.7275 - 2s/epoch - 12ms/step
Epoch 7/10
142/142 - 2s - loss: 0.0075 - accuracy: 0.9996 - val_loss: 0.7574 - val_accuracy: 0.7249 - 2s/epoch - 12ms/step
Epoch 8/10
142/142 - 2s - loss: 0.0043 - accuracy: 0.9998 - val_loss: 0.7967 - val_accuracy: 0.7302 - 2s/epoch - 17ms/step
Epoch 9/10
142/1